In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///home/conda/feedstock_root/build_artifacts/scikit-learn_1716489734259/work/dist/scikit_learn-1.5.0-cp311-cp311-linux_x86_64.whl#sha256=4e4d00ec549661aa22a03bd61e339cfd7abf80e247c75a0b3a95cc04b3c04b2d


In [2]:
!python -V

Python 3.11.0


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
year = 2023
month = 3
service_type = 'yellow' # green

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{service_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{service_type}/{year:04d}-{month:02d}.parquet'

df = read_data(input_file)

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
y_pred.std()

6.247488852238703

In [9]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [10]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predictions'] = y_pred.tolist()
df_result

,ride_id,predictions
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3403761,2023/03_3403761,11.952635
3403762,2023/03_3403762,20.049958
3403763,2023/03_3403763,11.595336
3403764,2023/03_3403764,13.113178


In [11]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
!ls -lah output/yellow/

total 66M
drwxr-xr-x 2 ellacharmed ellacharmed 4.0K Jun 16 00:46 .
drwxr-xr-x 3 ellacharmed ellacharmed 4.0K Jun 15 11:51 ..
-rw-r--r-- 1 ellacharmed ellacharmed  66M Jun 18 03:40 2023-03.parquet


In [13]:
# !docker build -t hmwk4:v1 . --no-cache

[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 308B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 308B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:mlops  0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 308B                                       0.0s


In [ ]:
# docker run --rm -it hmwk4:v1
# in main()
# read_parquet() done
# y_pred done
# std: 1.3881401513914737
# mean: 
# output file saved